In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from alpha import iterate_minibatches 
from alpha import make_square
from cnn_utils import conv_layer,fc_layer

In [136]:
import time, math
import matplotlib.pyplot as plt
import tensorflow as tf
import tqdm as tqdm
import os,cv2,random
import numpy as np
import pandas as pd
from collections import OrderedDict

%matplotlib inline

In [139]:
tf.__version__

'1.12.0'

In [4]:
num_data = 99
IMG_SIZE = 64
labels = []
data = []
DIR = "./train"
for i, im in enumerate(os.listdir(DIR)):
  if not im.startswith('.'):
    path = os.path.join(DIR, im)
    if i >= num_data : break
    img = cv2.imread(path)
    img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
    img = img/255.0
    data.append([np.array(img)])
    
    

In [5]:
train = np.array(data).reshape(-1,IMG_SIZE,IMG_SIZE,3)

In [6]:
train.shape

(99, 64, 64, 3)

In [7]:
np.save('./train.npy', train)

In [8]:
train = np.load('./train.npy').reshape(99, 64, 64, 3)

In [9]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15401533137564674239]

In [10]:
def saveGraph(graph):
  with tf.Session(graph = graph) as sess:
    filename = "./summary_log/VS-"+time.strftime("%H%M%S")
    writer = tf.summary.FileWriter(filename, sess.graph)
  print("Tensorboard summary saved to "+filename) midhun pk

In [11]:
graph1 = tf.Graph()

In [12]:
with graph1.as_default():
    X = tf.placeholder(tf.float32,[None,64,64,3])
    Y = tf.placeholder(tf.float32,[None,64,64,3])

In [13]:
def lrelu(b,alpha=0.1):
    return tf.maximum(alpha*b,b)

In [14]:
###Encoder

with graph1.as_default():
  with tf.name_scope('en-convolutions'):
    conv0 = tf.layers.conv2d(X, filters = 4, kernel_size  = (3, 3), strides = (1,1), padding = 'SAME', use_bias = True, activation = lrelu, name = 'conv0')
  # 32*32*4
  with tf.name_scope('en_pooling') :
    maxpool0 = tf.layers.max_pooling2d(conv0, pool_size = (2, 2), strides = (2, 2),name = 'pool0')
    
  with tf.name_scope('en-convolutions'):
    conv1 = tf.layers.conv2d(maxpool0, filters = 4, kernel_size  = (3, 3), strides = (1,1), padding = 'SAME', use_bias = True, activation = lrelu, name = 'conv1')
  
  # 32*32*4
  with tf.name_scope('en_pooling') :
    maxpool1 = tf.layers.max_pooling2d(conv1, pool_size = (2, 2), strides = (2, 2),name = 'pool0')
  # 16*16*4 
  with tf.name_scope('en-convolutions'):
    conv2 = tf.layers.conv2d(maxpool1, filters = 8, kernel_size  = (3, 3), strides = (1,1), padding = 'SAME', use_bias = True, activation = lrelu, name = 'conv2')
  # 16*16*8
  with tf.name_scope('en_pooling') :
    maxpool2 = tf.layers.max_pooling2d(conv2, pool_size = (2, 2), strides = (2, 2),name = 'pool0')
  # 8*8*8
  with tf.name_scope('en-convolutions'):
    conv3 = tf.layers.conv2d(maxpool2, filters = 16, kernel_size  = (3, 3), strides = (1,1), padding = 'SAME', use_bias = True, activation = lrelu, name = 'conv3')
  #  8x8x16
  with tf.name_scope('encoding'):
    encoded = tf.layers.average_pooling2d(conv3,pool_size=(2,2),strides=(2,2),name='encoding')
  

In [15]:
### decoder

with graph1.as_default():
  with tf.name_scope('decoder'):
    upsample1 = tf.layers.conv2d_transpose(encoded, filters = 16, kernel_size = 3, padding = 'SAME', strides = 2, name = 'upsample1')
    # 8*8*16
    conv4 = tf.layers.conv2d(upsample1, filters = 16, kernel_size = (3,3), strides = (1,1), padding = 'SAME', use_bias = True, name = 'conv4',activation =lrelu)
    # 8x8x16
    upsample2 = tf.layers.conv2d_transpose(conv4,filters=8,kernel_size=3,padding='same',strides=2,name='upsample2')
    # 16x16x8
    conv5 = tf.layers.conv2d(upsample2,filters=8,kernel_size=(3,3),strides=(1,1),name='conv5',padding='SAME',use_bias=True,activation=lrelu)
    # 16x16x8
    upsample3 = tf.layers.conv2d_transpose(conv5,filters=8,kernel_size=5,padding='same',strides=2,name='upsample3')
    # 32x32x8
    conv6 = tf.layers.conv2d(upsample3,filters=4,kernel_size=(5,5),strides=(1,1),name='conv6',padding='SAME',use_bias=True,activation=lrelu)
        
    # 16x16x8
    upsample4 = tf.layers.conv2d_transpose(conv6,filters=8,kernel_size=5,padding='same',strides=2,name='upsample4')
    # 32x32x8
    conv7 = tf.layers.conv2d(upsample4,filters=4,kernel_size=(5,5),strides=(1,1),name='conv7',padding='SAME',use_bias=True,activation=lrelu)
        
        
    # 32x32x4
    logits = tf.layers.conv2d(conv7,filters=3,kernel_size=(3,3),strides=(1,1),name='logits',padding='SAME',use_bias=True)
    #Now 32x32x3midhun pk
    # Pass logits through sigmoid to get reconstructed image
    decoded = tf.sigmoid(logits,name='recon')

In [16]:
saveGraph(graph1)midhun pk

Tensorboard summary saved to ./summary_log/VS-114909


In [17]:
with graph1.as_default():
  loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=Y)
  lr = tf.placeholder(tf.float32, shape=[])
  cost = tf.reduce_mean(loss) 
  opt = tf.train.AdamOptimizer(learning_rate=lr).minimize(cost) 

In [18]:
minibatch_size = 256
epoch = 20
learning_rate = 0.0005


with tf.Session(graph=graph1) as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    saver.restore(sess, tf.train.latest_checkpoint('./saved_model/'))
    for ep in range(epoch):
      avg_cost = 0
      for i, minibatch in enumerate(iterate_minibatches(train, minibatch_size)):
        batch_cost, _ = sess.run([cost, opt], feed_dict={X: minibatch,
                                                             Y: minibatch,
                                                            lr: learning_rate})
        
      print("Epoch: {}/{}...".format(ep+1, epoch), \
                  "Training loss: {:.4f}".format(batch_cost))
    saver.save(sess, "./saved_model/model")

INFO:tensorflow:Restoring parameters from ./saved_model/model
Epoch: 1/20... Training loss: 0.4881
Epoch: 2/20... Training loss: 0.4869
Epoch: 3/20... Training loss: 0.4857
Epoch: 4/20... Training loss: 0.4845
Epoch: 5/20... Training loss: 0.4833
Epoch: 6/20... Training loss: 0.4824
Epoch: 7/20... Training loss: 0.4814
Epoch: 8/20... Training loss: 0.4805
Epoch: 9/20... Training loss: 0.4798
Epoch: 10/20... Training loss: 0.4791
Epoch: 11/20... Training loss: 0.4784
Epoch: 12/20... Training loss: 0.4777
Epoch: 13/20... Training loss: 0.4769
Epoch: 14/20... Training loss: 0.4760
Epoch: 15/20... Training loss: 0.4751
Epoch: 16/20... Training loss: 0.4741
Epoch: 17/20... Training loss: 0.4732
Epoch: 18/20... Training loss: 0.4723
Epoch: 19/20... Training loss: 0.4714
Epoch: 20/20... Training loss: 0.4705


In [19]:
limit = 9999
DIR = './train'
IMG_SIZE = 64
embeddings = []
labels = []
with tf.Session(graph=graph1) as sess:
    saver = tf.train.Saver()
    saver.restore(sess, tf.train.latest_checkpoint('./saved_model/'))
    for i, im in enumerate(os.listdir(DIR)):    
        if not im.startswith('.'):
            path = os.path.join(DIR, im)
            if i >= limit: break
            img = cv2.imread(path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
            img = img/255.0
            embeddings.append(sess.run(encoded,feed_dict={X:img.reshape((-1,64,64,3))}).reshape(-1,256))
            labels.append(int(path.split('.')[-2].split('-')[-1]))
labels = np.array(labels)
embeddings = np.array(np.squeeze(embeddings))

INFO:tensorflow:Restoring parameters from ./saved_model/model


In [20]:
# Saving npy for future usage
np.save('./embeddings.npy', embeddings)
np.save('./labels.npy', labels)

In [21]:
embeddings = np.load('./embeddings.npy').reshape(5000, 256) 
labels = np.load('./labels.npy').reshape(5000,)

In [22]:
data = pd.DataFrame(embeddings)
data['labels'] = labels
data.set_index('labels', inplace=True)
data.sort_index(inplace=True)


In [23]:
np.save('./data.npy', embeddings)
data = np.load('./data.npy').reshape(5000, 256)
data = pd.DataFrame(data)

In [24]:
Data = pd.read_csv('./Data/data.csv')
tunicSlice = pd.read_csv('./Data/tunics.csv')

def getinfo(idx):
  pid = Data[Data.index==idx]['productId'].values[0]
  idxTunic = tunicSlice[tunicSlice.productId==pid]['index'].values[0]
  return idxTunic, tunicSlice.loc[idxTunic]['sellerName']

In [25]:
def getTestimage(idx):
  
  from PIL import Image
  img = path = './train/im-400-{}.jpg'.format(idx)
  img = Image.open(path)
  img = make_square(img)
  img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

In [107]:
def getEmbeddings(idx):
  
  img = IMG_SIZE = 64
  img = cv2.imread('./train/im-400-{}.jpg'.format(idx))
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
  img = img/255.0
  
  with tf.Session(graph=graph1) as sess:
    saver = tf.train.Saver()
    saver.restore(sess, tf.train.latest_checkpoint('./saved_model/'))
    dist = sess.run(encoded,feed_dict={X:img.reshape((-1,64,64,3))})
    
  return np.squeeze(dist.reshape(-1,4*4*16))
  

In [135]:
img = IMG_SIZE = 64
img = cv2.imread('./train/im-400-{}.jpg'.format(1))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
img = img/255.0

with tf.Session(graph=graph1) as sess:
  saver = tf.train.Saver()
  saver.restore(sess, tf.train.latest_checkpoint('./saved_model/'))
  dist = sess.run(encoded,feed_dict={X:img.reshape((-1,64,64,3))})
  x = np.squeeze(dist.reshape(-1,256))
  print(x)

INFO:tensorflow:Restoring parameters from ./saved_model/model
[ 0.31782973 -0.01172512  0.17890753 -0.04678841  0.48739654  0.10384879
  0.25308782  0.46912694  0.47205538 -0.06895326  0.15418787  0.04846708
 -0.07288961 -0.01407217 -0.02208111  0.16211846  0.35695687 -0.01479566
  0.14447856 -0.02823993  0.36247307  0.03484306  0.22247162  0.26179254
  0.4566845  -0.04280451  0.04151934  0.10462055 -0.06762135 -0.00361139
 -0.01558663  0.22934838  0.30016872 -0.01910256  0.11116482 -0.04157083
  0.46055123  0.05664933  0.21092622  0.4214196   0.4343406  -0.05453917
  0.11362436  0.01554032 -0.07284284 -0.01475112 -0.02215968  0.21870632
  0.31301776 -0.01696425  0.23367606 -0.04322098  0.44799134  0.07404392
  0.40627357  0.5303667   0.59851325 -0.05248134  0.05909903  0.04067545
 -0.05752292 -0.01961535 -0.02448858  0.08952505  0.17008445 -0.02305754
  0.21171992 -0.04065169  0.31459963 -0.04194951  0.14529046  0.3243929
  0.3437604  -0.05206476  0.02951137  0.05871679 -0.05638153 -0

In [28]:
def getPredictedImages(score):
  images = []
  IMG_SIZE = 200
  
  for idx in score.index:
    path = './train/im-400-{}.jpg'.format(1)
    img = Image.open(path)
    img  = make_square(img)
    img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
    images.append([np.array(img)])
  return np.array(images).reshape(-1, 200,200,3)

In [57]:
def getPredictions(testEmbeddings):
  dist = (np.array(data) - testEmbeddings)**2
  dist = np.sum(dist, axis=1)
  dist = np.sqrt(dist)
  
  df = pd.DataFrame({'distance':dist})
  df.index = data.index
  df.sort_values('distance', ascending=True, inplace=True)
  score = df[:20]
  score['score'] = df.distance[:20].apply(lambda x: np.round(1-np.tanh(x)**10, 30)).values
  return score

In [112]:
def plotImages(idx, im, images, score):
  fig, axes = plt.subplots(5,2, figsize = (15,25))
  fig.subplots_adjust(hspace = 0.3, wspace = 0.3)
  for i, ax in enumerate(axes.flat):
    if i==0:
      idX, sellerName = getinfo(idx)
      ax.imshow(im, cmap = 'binary')
      xlabel = "Original image \n id: {} \n Seller: {}".format(idx, sellerName)
      ax.set_xlabel(xlabel, fontsize = 13)
    else:
      ax.imshow(images[i-1], cmap = 'binary')
      idX, sellerName = getinfo(score.index[i])
      xlabel = "score: {} \nid:{} \n Seller: {}".format(score.score.iloc[i-1], idX, sellerName)
      ax.set_xlabel(xlabel, fontsize = 12)
      #Remove axis ticks
      ax.set_xticks([])
      ax.set_yticks([])
      
      
  plt.show()

In [115]:
out = dict()

In [123]:
sc = []
for i in range(2):
  testEmbeddings = getEmbeddings(1)
  score = getPredictions(testEmbeddings)
  ls = [[score.index[i], score.score.iloc[i]] for i in range(20) if score.score.iloc[i]>0.89]
  ind, _ = getinfo(i)
  out[str(ind)] = ls

INFO:tensorflow:Restoring parameters from ./saved_model/model
INFO:tensorflow:Restoring parameters from ./saved_model/model


In [124]:
idx = 1993
im, testEmbeddings = getEmbeddings(idx)
images, score = getPredictions(testEmbeddings)
plotImages(idx, im, images, score)

INFO:tensorflow:Restoring parameters from ./saved_model/model


ValueError: too many values to unpack (expected 2)